# Choral Explanations on Alkemio

At the [TIP knowledge base](https://alkem.io/tip/knowledge-base) on [Alkemio](https://welcome.alkem.io/) we apply [Choral Explanations](https://hapgood.us/2016/05/13/choral-explanations/) to develop knowledge commons in a Q&A format. In this project we explore new possibilities for navigation and visualisation.

## Copyright and license

### Code

Copyright: sander.dijkhuis@cleverbase.com 2024; Licensed under the EUPL.

### Example content

See content source.

In [46]:
import json
import os
import urllib
from datetime import datetime
from urllib.request import Request
from IPython.display import display, Markdown

os.environ["no_proxy"] = "*"

Use the [GraphQL Playground](https://alkem.io/graphql) to develop queries.

In [65]:
endpoint = "https://alkem.io/api/private/graphql"
reader = open("queries.graphql", "r")
queries = reader.read()
reader.close()

In [3]:
def query(operation, variables):
    payload = {
        "query": queries,
        "operationName": operation,
        "variables": variables
    }
    headers = {
        "Content-Type": "application/json"
    }
    body = json.dumps(payload).encode("utf-8")
    response = urllib.request.urlopen(Request(endpoint, body, headers))
    return json.loads(response.read().decode("utf-8"))

In [67]:
result = query("questions", { "spaceNameId": "tip" })

In [68]:
result

{'data': {'space': {'collaboration': {'callouts': [{'nameID': 'watvoorinitiatieve-1713',
      'type': 'POST_COLLECTION',
      'createdBy': {'profile': {'url': 'https://alkem.io/user/sander-boer-499',
        'displayName': 'Sander Boer'}},
      'framing': {'profile': {'url': 'https://alkem.io/tip/collaboration/watvoorinitiatieve-1713',
        'displayName': 'Wat voor initiatieven zijn er voor de inzet van datakluizen in Nederland en wat is de impact op TIP?',
        'description': 'Als architect wil ik weten wat voor initiatieven er zijn voor de inzet van datakluizen / persoonlijke dataspaces (bijv. Solid Pod) in Nederland en wat is de impact is op TIP, zodat ik passende kennisbronnen en samenwerkingen kan opzoeken.\n\n*Bijdragen zijn gelicenseerd onder *[*CC BY 4.0*](https://creativecommons.org/licenses/by/4.0/deed.nl)*.*\n'}},
      'contributions': [{'post': {'nameID': 'solidcommunity-2317',
         'createdBy': {'profile': {'url': 'https://alkem.io/user/sander-dijkhuis-3912',

In [73]:
output = []

for question in result["data"]["space"]["collaboration"]["callouts"]:
    title = question["framing"]["profile"]["displayName"]
    url = question["framing"]["profile"]["url"]
    output.append(f"## [{title}]({url})\n\n")
    author = question["createdBy"]["profile"]["displayName"]
    author = "Vrager"
    url = question["createdBy"]["profile"]["url"]
    output.append(f"Oorspronkelijk gevraagd door [{author}]({url})\n\n")
    description = question["framing"]["profile"]["description"].splitlines()
    for line in description:
        output.append(f"> {line}\n")
    output.append("\n\n")
    for answer in question["contributions"]:
        title = answer["post"]["profile"]["displayName"]
        url = answer["post"]["profile"]["url"]
        output.append(f"### [{title}]({url})\n\n")
        author = answer["post"]["createdBy"]["profile"]["displayName"]
        author = "Antwoorder"
        url = answer["post"]["createdBy"]["profile"]["url"]
        output.append(f"Oorspronkelijk geschreven door [{author}]({url})\n\n")
        description = answer["post"]["profile"]["description"].splitlines()
        for line in description:
            output.append(f"> {line}\n")
        output.append("\n\n")
        for comment in answer["post"]["comments"]["messages"]:
            author = comment["sender"]["profile"]["displayName"]
            author = "Reageerder"
            url = comment["sender"]["profile"]["url"]
            time = datetime.fromtimestamp(comment["timestamp"] / 1000)
            date = time.strftime("%Y-%m-%d %H:%M UTC")
            message = comment["message"].splitlines()
            output.append(f"#### [{author}]({url}) {date}\n\n")
            for line in message:
                output.append(f"> {line}\n")
            output.append("\n\n")

display(Markdown("".join(output)))

## [Wat voor initiatieven zijn er voor de inzet van datakluizen in Nederland en wat is de impact op TIP?](https://alkem.io/tip/collaboration/watvoorinitiatieve-1713)

Oorspronkelijk gevraagd door [Vrager](https://alkem.io/user/sander-boer-499)

> Als architect wil ik weten wat voor initiatieven er zijn voor de inzet van datakluizen / persoonlijke dataspaces (bijv. Solid Pod) in Nederland en wat is de impact is op TIP, zodat ik passende kennisbronnen en samenwerkingen kan opzoeken.
> 
> *Bijdragen zijn gelicenseerd onder *[*CC BY 4.0*](https://creativecommons.org/licenses/by/4.0/deed.nl)*.*


### [Solid Community](https://alkem.io/tip/collaboration/watvoorinitiatieve-1713/posts/solidcommunity-2317)

Oorspronkelijk geschreven door [Antwoorder](https://alkem.io/user/sander-dijkhuis-3912)

> Hierover spraken we tijdens de kennissessie van 10 november 2023 met de vertegenwoordiger van Solid Community Foundation. Deze stichting wil de implementatie van Solid voor datakluizen bevorderen. Zie ook: [Wat is Solid?](https://alkem.io/tip/collaboration/watissolid-6045) Er kwamen meerdere Nederlandse initiatieven aan bod zoals de Nederlandse Datakluis ([Wat is de Nederlandse Datakluis?](https://alkem.io/tip/collaboration/watisdenederlands-7532)) het Virtueel Inkomsten Loket.


## [Welke randvoorwaarden zijn nodig zodat Nederlanse relying parties  id-wallets kunnen accepteren?](https://alkem.io/tip/collaboration/welkerandvoorwaarde-6229)

Oorspronkelijk gevraagd door [Vrager](https://alkem.io/user/sander-boer-499)

> Als er straks een edi wallet bestaat dan moet aan de kant van dienstverleners ook nog wat gebeuren om die te kunnen (en mogen) te accepteren. Aan welke voorwaarden moeten die voldoen? Komen die dienstverleners onder een vorm van toezicht te vallen om bijvoorbeeld overvraging te voorkomen en dat ze de doelbinding kunnen aantonen, etc..
> 
> <br>
> 
> *Bijdragen zijn gelicenseerd onder *[*CC BY 4.0*](https://creativecommons.org/licenses/by/4.0/deed.nl)*.*


## [Welke wallets zijn beschikbaar voor implementatie in TIP?](https://alkem.io/tip/collaboration/welkewalletszijnb-9243)

Oorspronkelijk gevraagd door [Vrager](https://alkem.io/user/sander-dijkhuis-3912)

> Als TIP-partner wil ik weten met welke wallets ik rekening kan houden voor implementatie van TIP-basisfuncties.
> 
> *Bijdragen zijn gelicenseerd onder *[*CC BY 4.0*](https://creativecommons.org/licenses/by/4.0/deed.nl)*.*


### [Digital Wallet and Agent Overviews van Open Wallet Foundation](https://alkem.io/tip/collaboration/welkewalletszijnb-9243/posts/digitalwalletanda-2218)

Oorspronkelijk geschreven door [Antwoorder](https://alkem.io/user/sander-dijkhuis-3912)

> Op initiatief van TNO houdt de Open Wallet Foundation een overzicht bij:
> 
> [Digital Wallet and Agent Overviews Special Interest Group](https://github.com/openwallet-foundation/digital-wallet-and-agent-overviews-sig)
> 
> Hierin ontbreken momenteel nog wallets van QTSPs, en is niet duidelijk welke al geschikt zijn voor de basisfuncties van TIP. Deelnemers kunnen voorstellen voor wijzigingen doen om dit toe te voegen.


## [Kan een bank SCA delegeren aan een QTSP?](https://alkem.io/tip/collaboration/kaneenbankscadel-5381)

Oorspronkelijk gevraagd door [Vrager](https://alkem.io/user/sander-dijkhuis-3912)

> Als bank wil ik weten in hoeverre ik eisen aan *strong customer authentication* (SCA) kan delegeren aan een gekwalificeerd vertrouwensdienstverlener onder de eIDAS, met het oog op hergebruik van wallets en eventuele verplichtingen uit de herziening van eIDAS.
> 
> *Bijdragen zijn gelicenseerd onder *[*CC BY 4.0*](https://creativecommons.org/licenses/by/4.0/deed.nl)*.*


### [Toepassing van gekwalificeerde certificaten en beperking van aansprakelijkheid](https://alkem.io/tip/collaboration/kaneenbankscadel-5381/posts/toepassingvangekwa-1586)

Oorspronkelijk geschreven door [Antwoorder](https://alkem.io/user/sander-dijkhuis-3912)

> [IANAL](https://en.wikipedia.org/wiki/IANAL) maar bij deze wat ik begrijp op basis van een aantal gesprekken.
> 
> Banken zijn onder PSD2 Artikel 97(1) verplicht om SCA uit te voeren bij toegang, initiatie betaaltransacties, en andere riskante online acties. In Artikel 4(3) definieert PSD2 het concept SCA als multi-factor authenticatie van de gebruiker.
> 
> Let op: SCA is onderdeel van een set aan risicogebaseerde maatregelen, en niet de kapstok waaronder ook alle andere risicogebaseerde maatregelen bij transacties vallen. Dus de vraag over SCA met een QTSP gaat niet over het geheel verleggen van de verantwoordelijkheid voor het maatregelenpakket, alleen over het delegeren van uitgifte en beheer van een authenticatiemiddel.
> 
> Een gekwalificeerd certificaat (QC) is zo’n authenticatiemiddel, en wordt in de praktijk vaak aan een wallet uitgegeven. Daarbij aanvaardt de uitgevende QTSP aansprakelijkheid bij fouten in naleving van eIDAS bij identiteitsverificatie. De QTSP mag die aansprakelijkheid beperken. De beperking zal vaak samenhangen met de omzet die de QTSP maakt op die dienstverlening.
> 
> Als de bank het risico op identiteitsfraude als hoog beoordeelt, is een mogelijke maatregel om een relatief vers QC te vereisen. Zo beperkt de bank het risico dat de wallet na uitgifte in handen is gekomen van een andere persoon. Er zijn ook andere maatregelen te bedenken.
> 
> Stel de bank voert een betaaltransactie van €100M uit op basis van een vers QC uitgegeven door een QTSP die aansprakelijkheid beperkt tot €100k. Wat als de identiteit alsnog niet blijkt te kloppen, door het onvoldoende naleven van eIDAS door de QTSP, en daarmee de €100M naar de verkeerde rekening gaat?
> 
> Nu is het nodig om naar twee bedragen te kijken:
> 
> *   Onterecht verplaatst geld: €100M. Hier ligt voor de hand dat banken en/of partijen in de transactie aanvullende maatregelen treffen bovenop SCA. Denk aan verzekeringen of aanvullende procedures.
> *   Schade: denk aan herstelschade, bijvoorbeeld om de fraudeur te bereiken, en verminderde goodwill of winsten.
> 
> In eIDAS is de aansprakelijkheid voor de schade rond vertrouwensdienstverlening geregeld. Deze zal in de praktijk veel lager liggen dan de onterecht verplaatste €100M. Daarmee lijkt de beperking in aansprakelijkheid door QTSPs geen barrière.


#### [Reageerder](https://alkem.io/user/sander-dijkhuis-3912) 2024-01-10 18:56 UTC

> Zie ook deze gerelateerde vraag: [Wat moet er geregeld worden rond de aansprakelijkheid zodra een PID geaccepteerd moet worden?](https://alkem.io/tip/collaboration/watmoetergeregeld-7715).


#### [Reageerder](https://alkem.io/user/nitesh-bharosa-5829) 2024-01-15 15:47 UTC

> 👍


## [Wat is de toegevoegde waarde van vertrouwensdiensten voor de digitale overheid?](https://alkem.io/tip/collaboration/watisdetoegevoegd-5977)

Oorspronkelijk gevraagd door [Vrager](https://alkem.io/user/sander-dijkhuis-3912)

> Als medewerker van een beleidsdepartement of uitvoeringsorganisatie wil ik weten waar kan worden geleund op gekwalificeerde certificaten en andere voorzieningen uit gekwalificeerde vertrouwensdiensten.
> 
> *Bijdragen zijn gelicenseerd onder *[*CC BY 4.0*](https://creativecommons.org/licenses/by/4.0/deed.nl)*.*


